In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from scipy.spatial import Delaunay
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.pyplot as plt
from matplotlib import cm

# Mediapipe Face Mesh 모듈 초기화
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# 얼굴 랜드마크 좌표를 추출하는 함수
def extract_face_landmarks(image):
    with mp_face_mesh.FaceMesh(min_detection_confidence=0.1, min_tracking_confidence=0.1) as face_mesh:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image_rgb)

        landmarks = []
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                for landmark in face_landmarks.landmark:
                    landmarks.append((landmark.x, landmark.y, landmark.z))
        return landmarks

# 얼굴 랜드마크 좌표를 OBJ 형식으로 저장하는 함수
def save_landmarks_to_obj_with_texture_smooth(landmarks, obj_filename, texture_filename, image):
    # 얼굴 랜드마크 좌표를 NumPy 배열로 변환
    vertices = np.array(landmarks)

    # Delaunay 삼각화
    tri = Delaunay(vertices[:, :2])

    # 텍스처 이미지 생성
    texture_image = np.zeros_like(image)
    for simplex in tri.simplices:
        cv2.fillPoly(texture_image, [vertices[simplex, :2].astype(int)], color=(255, 255, 255))

    # 텍스처 이미지 저장
    cv2.imwrite(f"{texture_filename}.jpg", texture_image)

    # OBJ 파일 작성
    with open(obj_filename, 'w') as obj_file:
        # 텍스처 매핑 관련 정보를 추가
        obj_file.write(f"mtllib {texture_filename}.mtl\n")
        obj_file.write(f"usemtl {texture_filename}\n")

        for vertex, simplex in zip(landmarks, tri.simplices):
            obj_file.write(f'v {vertex[0]} {vertex[1]} {vertex[2]}\n')
            obj_file.write(f'vt {vertex[0]} {vertex[1]}\n')  # 텍스처 좌표 추가

        # 세부적인 삼각화와 보간을 위한 추가 정점 생성
        for simplex in tri.simplices:
            midpoints = np.mean(vertices[simplex], axis=0)
            obj_file.write(f'v {midpoints[0]} {midpoints[1]} {midpoints[2]}\n')
            obj_file.write(f'vt {midpoints[0]} {midpoints[1]}\n')

        # 추가 정점을 활용하여 부드러운 면 생성
        for i in range(tri.nsimplex):
            obj_file.write(f'f {tri.simplices[i, 0]+1}/{tri.simplices[i, 0]+1} '
                           f'{tri.simplices[i, 1]+1}/{tri.simplices[i, 1]+1} '
                           f'{tri.simplices[i, 2]+1}/{tri.simplices[i, 2]+1}\n')
            obj_file.write(f'f {tri.simplices[i, 0]+1}/{tri.simplices[i, 0]+1} '
                           f'{tri.simplices[i, 1]+1}/{tri.simplices[i, 1]+1} '
                           f'{tri.npoints + i + 1}/{tri.npoints + i + 1}\n')
            obj_file.write(f'f {tri.simplices[i, 1]+1}/{tri.simplices[i, 1]+1} '
                           f'{tri.simplices[i, 2]+1}/{tri.simplices[i, 2]+1} '
                           f'{tri.npoints + i + 1}/{tri.npoints + i + 1}\n')
            obj_file.write(f'f {tri.simplices[i, 2]+1}/{tri.simplices[i, 2]+1} '
                           f'{tri.simplices[i, 0]+1}/{tri.simplices[i, 0]+1} '
                           f'{tri.npoints + i + 1}/{tri.npoints + i + 1}\n')

    # 텍스처 매핑 정보를 저장하는 MTL 파일 작성
    with open(f"{texture_filename}.mtl", 'w') as mtl_file:
        mtl_file.write(f"newmtl {texture_filename}\n")
        mtl_file.write("Ka 1.000 1.000 1.000\n")
        mtl_file.write("Kd 1.000 1.000 1.000\n")
        mtl_file.write(f"map_Kd {texture_filename}.jpg\n")  # 텍스처 이미지 파일명 추가

# 이미지 로드
image_path = 'test_img_1.jpg'
image = cv2.imread(image_path)

# 얼굴 랜드마크 추출
face_landmarks = extract_face_landmarks(image)

# 얼굴 랜드마크를 OBJ 파일로 저장 (텍스처 매핑 및 부드러운 모델 추가)
if face_landmarks:
    obj_filename = 'out_smooth.obj'
    texture_filename = 'out_smooth'
    save_landmarks_to_obj_with_texture_smooth(face_landmarks, obj_filename, texture_filename, image)
    print(f'Successfully saved the 3D mesh to {obj_filename} with texture mapping and smooth shading.')
else:
    print('No face landmarks detected.')
